In [ ]:
# requried libs
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("day.csv")

1. DATA Exploration

In [ ]:
data.head(6)

In [ ]:
data.columns

In [ ]:
data.describe()

In [ ]:
data.info()

In [4]:
# asigning string to numeric values
data.loc[data['season'] == 1, 'season'] = 'spring'
data.loc[data['season'] == 2, 'season'] = 'summer'
data.loc[data['season'] == 3, 'season'] = 'fall'
data.loc[data['season'] == 4, 'season'] = 'winter'

In [7]:
data['season'].astype('category').value_counts()

season
fall      188
summer    184
spring    180
winter    178
Name: count, dtype: int64

In [ ]:
def chg_mnth(x):
    return x.map({1: 'jan', 2:'feb', 3:'mar', 4:'apr', 5:'may', 6:'jun', 7:'jul', 8:'aug', 9:'sep', 10:'oct', 11:'nov', 12:'dec'})

In [ ]:
data[['mnth']] = data[['mnth']].apply(chg_mnth)

In [ ]:
def chg_weekday(x):
    return x.map({1: 'Mon', 2:'Tue', 3:'wed', 4:'Thu', 5:'Fri', 6:'Sat', 0:'Sun'})

In [ ]:
data[['weekday']] = data[['weekday']].apply(chg_weekday)

In [ ]:
data['holiday'].astype('category').value_counts()

In [ ]:
data.loc[data['weathersit'] == 1, 'weathersit'] = 'A'
data.loc[data['weathersit'] == 2, 'weathersit'] = 'B'
data.loc[data['weathersit'] == 3, 'weathersit'] = 'C'

In [ ]:
data['weathersit'].astype('category').value_counts()

2. Data Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.distplot(data['temp'])

In [ ]:
sns.distplot(data['atemp'])

In [ ]:
sns.distplot(data['windspeed'])

In [ ]:
sns.distplot(data['cnt'])

In [ ]:
# convert date into datetime format
data['dteday'] = data['dteday'].astype('datetime64[s]')

In [ ]:
data.info()

In [ ]:
data_catageries = data.select_dtypes(exclude=['float64', 'datetime64', 'int64'])

In [ ]:
data_catageries

In [ ]:
plt.figure(figsize=(20,20))
plt.subplot(3, 3, 1)
sns.boxplot(x='yr', y='cnt', data=data)
plt.subplot(3, 3, 2)
sns.boxplot(x='mnth', y='cnt', data=data)
plt.subplot(3, 3, 3)
sns.boxplot(x='weekday', y='cnt', data=data)
plt.subplot(3, 3, 4)
sns.boxplot(x='workingday', y='cnt', data=data)
plt.subplot(3, 3, 5)
sns.boxplot(x='season', y='cnt', data=data)
plt.subplot(3, 3, 6)
sns.boxplot(x='holiday', y='cnt', data=data)
plt.subplot(3, 3, 7)
sns.boxplot(x='weathersit', y='cnt', data=data)
plt.show()

In [ ]:
data.info()

In [ ]:
intvar = ['casual', 'registered', 'cnt']
for var in intvar:
    data[var] = data[var].astype('float64')

In [ ]:
data_numeric = data.select_dtypes('float64')
data_numeric.head()

In [ ]:
# pairplot are graphs with each other 
sns.pairplot(data_numeric)
plt.show()

In [ ]:
corr = data_numeric.corr()
corr

In [ ]:
# heapmap of data_numeric
mask = np.array(corr)
mask[np.tril_indices_from(mask)] = False

fig, ax = plt.subplots()
fig.set_size_inches(10, 10)

sns.heatmap(corr, mask=mask, vmax=1, square=True, annot=True)

In [ ]:
data.drop('atemp', axis=1, inplace=True)

In [ ]:
data.info()

3. DATA Preparation 

In [ ]:
data.info()

In [ ]:
data_catageries = data.select_dtypes(include=['object'])
data_catageries

In [ ]:
data_dummy = pd.get_dummies(data_catageries, drop_first= True, dtype='int')

In [ ]:
data_dummy.head()

In [ ]:
# making a set of data other than catageries for the modal
data = data.drop(list(data_catageries.columns)+['instant', 'dteday'], axis=1)

In [ ]:
fdata = pd.concat([data, data_dummy], axis=1)

In [ ]:
fdata.head()

4. Model Building & Evaluating

In [ ]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
#split train and test datasets
np.random.seed(0)
df_train, df_test = train_test_split(fdata, train_size=0.7, test_size=0.3, random_state=100)

In [ ]:
df_train

In [ ]:
# applying scalar to columns
from sklearn.preprocessing import MinMaxScaler
scalar = MinMaxScaler()
var = ['temp', 'hum', 'windspeed', 'casual', 'registered', 'cnt']
df_train[var] = scalar.fit_transform(df_train[var])

In [ ]:
plt.figure(figsize= (30, 30))    
sns.heatmap(df_train.corr(), annot=True, cmap="YlGnBu")
plt.show()

In [ ]:
x_train = df_train.drop(['casual', 'registered'], axis = 1)
y_train = df_train.pop('cnt')

In [ ]:
# Training data 
import statsmodels.api as sm
x_train_lm = sm.add_constant(x_train)

lr = sm.OLS(y_train,x_train_lm).fit()

In [ ]:
lr.params

In [ ]:
LR = LinearRegression()

LR.fit(x_train, y_train)

In [ ]:
print(LR.coef_)
print(LR.intercept_)

In [ ]:
lr.summary()

In [ ]:
# removisng some features from the data which has less influence
from sklearn.feature_selection import RFE 

rfe1 = RFE(estimator=LR, n_features_to_select=15)
rfe1.fit(x_train, y_train)
print(rfe1.support_)
print(rfe1.ranking_)

In [ ]:
col1 = x_train.columns[rfe1.support_]

In [ ]:
col1

In [ ]:
x_train_rfe1 = x_train[col1]
x_train_rfe1 = sm.add_constant(x_train_rfe1)
lm = sm.OLS(y_train,x_train_lm).fit()
lm.summary()

In [ ]:
# evaluating VIFs
from statsmodels.stats.outliers_influence import variance_inflation_factor

a = x_train_rfe1.drop('const', axis=1)

In [ ]:
vif = pd.DataFrame()
vif['features'] = a.columns
vif['VIF'] = [ variance_inflation_factor(a.values, i) for i in range(a.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by= 'VIF', ascending=False)
vif

In [ ]:
lm = LinearRegression()
rfe2 = RFE(estimator=LR, n_features_to_select=7)

# fit with 7 features
rfe2.fit(x_train, y_train)
print(rfe2.support_)
print(rfe2.ranking_)

In [ ]:
col2 = x_train.columns[rfe2.support_]

x_train_rfe2 = x_train[col2]
x_train_rfe2 = sm.add_constant(x_train_rfe2)
lm2 = sm.OLS(y_train,x_train_rfe2).fit()
lm2.summary()

In [ ]:
b = x_train_rfe2.drop('const', axis=1)
vif1 = pd.DataFrame()
vif1['features'] = b.columns
vif1['VIF'] = [ variance_inflation_factor(b.values, i) for i in range(b.shape[1])]
vif1['VIF'] = round(vif1['VIF'], 2)
vif1 = vif1.sort_values(by= 'VIF', ascending=False)
vif1

In [ ]:
df_test[var] =scalar.transform(df_test[var])
df_test

In [ ]:
x_test = df_test.drop(['casual', 'registered'], axis = 1)
y_test = df_test.pop('cnt')

In [ ]:
x_test

In [ ]:
c = x_train_rfe2.drop('const', axis=1)
c.columns

In [ ]:
x_test_rfe2 = x_test[col2]
x_test_rfe2 = sm.add_constant(x_test_rfe2)
x_test_rfe2.info()

In [ ]:
y_pred = lm2.predict(x_test_rfe2)

In [ ]:
plt.figure()
plt.scatter(y_test, y_pred)

In [ ]:
from sklearn.metrics import r2_score 
r2_score(y_test, y_pred)

In [ ]:
plt.figure(figsize=(8,5))

sns.heatmap(data[col2].corr(), cmap='YlGnBu', annot=True)
plt.show()